In [38]:
import numpy as np
import torch
import os
import cv2
import torch
import pandas as pd

DB_path = './data/ex'
output_shape = (416, 416)

label_csv = pd.read_csv('ex_train.csv')
labels = torch.arange(20)
one_hot = torch.nn.functional.one_hot(labels)

for idx in range(1):
    img_path = os.path.join(f'{DB_path}', label_csv.iloc[idx, 0])
    image = cv2.resize(cv2.imread(img_path), output_shape) / 255
    print(type(image))
    image = torch.from_numpy(image).permute(2, 0, 1).float()
    print(type(image))
    print(image)

    sizes = label_csv.iloc[idx, 1]
    sizes = list(map(int, sizes.split(' ')))

    bboxs = label_csv.iloc[idx, 2]
    print(type(bboxs))
    bboxs = list(map(str, bboxs.split(',')))

    bbox = []
    for bx in bboxs:
        normal = list(map(int, bx.split(' ')))
        xmin = normal[0] / sizes[0]
        ymin = normal[1] / sizes[1]
        xmax = normal[2] / sizes[0]
        ymax = normal[3] / sizes[1]

        bbox.append([xmin, ymin, xmax, ymax])
    bbox = np.array(bbox)
    print(bbox)

    clss = str(label_csv.iloc[idx, 3])
    clss = list(map(int, clss.split(',')))

    cls = one_hot[clss]
    cls = np.array(cls)




# labels = torch.arange(20).view(-1, 1)
# print(labels)
# one_hot = torch.zeros(20, 20).scatter_(1, labels, 1)
# print(one_hot)
#
# one_hot = torch.FloatTensor(20, 20).zero_()
# target = one_hot.scatter_(1, labels.data, 1)
# print(target)

<class 'numpy.ndarray'>
<class 'torch.Tensor'>
tensor([[[0.9882, 0.7843, 0.5333,  ..., 0.7412, 0.7216, 0.6314],
         [1.0000, 0.7451, 0.4980,  ..., 0.8863, 0.8667, 0.8235],
         [1.0000, 0.6863, 0.4275,  ..., 0.8510, 0.8431, 0.8235],
         ...,
         [0.2000, 0.2667, 0.3373,  ..., 0.5922, 0.6000, 0.6118],
         [0.3137, 0.3333, 0.3686,  ..., 0.5882, 0.5922, 0.6000],
         [0.3490, 0.3608, 0.3412,  ..., 0.6000, 0.6039, 0.6118]],

        [[0.4471, 0.2863, 0.1216,  ..., 0.8078, 0.7843, 0.6980],
         [0.5451, 0.3020, 0.1216,  ..., 0.9294, 0.9059, 0.8667],
         [0.6235, 0.3059, 0.0941,  ..., 0.8706, 0.8627, 0.8471],
         ...,
         [0.2588, 0.3255, 0.3922,  ..., 0.7059, 0.7137, 0.7255],
         [0.3608, 0.3804, 0.4235,  ..., 0.6980, 0.7059, 0.7216],
         [0.3922, 0.4118, 0.3961,  ..., 0.7137, 0.7216, 0.7333]],

        [[0.2392, 0.1176, 0.0314,  ..., 0.8510, 0.8275, 0.7333],
         [0.3647, 0.1529, 0.0314,  ..., 0.9686, 0.9451, 0.8980],
         [0

In [134]:
import cv2
import torch
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

anchors_wh = torch.tensor([[10, 13], [16, 30], [33, 23],
                           [30, 61], [62, 45], [59, 119],
                           [116, 90], [156, 198], [373, 326]]).float() / 416

DB_path = './data/VOC2007_trainval'
csv_file = '2007_train.csv'

class CustomDataset(Dataset):
    def __init__(self, num_classes, output_shape=(416, 416)):
        self.label_csv = pd.read_csv(f'{csv_file}')
        self.num_classes = num_classes
        self.output_shape = output_shape

    def __len__(self):
        return len(self.label_csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(f'{DB_path}', self.label_csv.iloc[idx, 0])
        image = cv2.resize(cv2.imread(img_path), self.output_shape) / 255
        image = torch.from_numpy(image).permute(2, 0, 1).float()

        bboxes, classes = self.parse_y_features(idx)
        label = (
            self.preprocess_label_for_one_scale(bboxes, classes,
                                                52, torch.tensor([0, 1, 2]).cuda()),
            self.preprocess_label_for_one_scale(bboxes, classes,
                                                26, torch.tensor([3, 4, 5]).cuda()),
            self.preprocess_label_for_one_scale(bboxes, classes,
                                                13, torch.tensor([6, 7, 8]).cuda())
        )
        return image, label

    def preprocess_label_for_one_class(self, bbox, classes, grid_size, valid_anchor):
        y = torch.zeros((grid_size, grid_size, 3, 5 + self.num_classes))
        anchor_idx = self.find_best_anchor(bbox)

        num_boxes = classes.shape[0]
        for i in range(num_boxes):
            curr_class = torch.tensor(classes[i]).float()
            curr_box = torch.tensor(bbox[i]).float()
            curr_anchor = anchor_idx[i]

            # 앵커는 현재 그리드 사이즈에 맞는 것만 사용, 인덱스 3개로 조정
            if curr_anchor in valid_anchor:
                adjusted_anchor_idx = curr_anchor % 3

                # loss 계산을 위해 (xmin, ymin, xmax, ymax)를 (x, y, w, h)로 변환
                curr_box_xy = (curr_box[..., 0:2] + curr_box[..., 2:4]) / 2
                curr_box_wh = curr_box[..., 2:4] - curr_box[..., 0:2]

                # grid cell의 index
                grid_cell_xy = curr_box_xy // float(1 / grid_size)
                # gird[y][x][anchor] 형태 = (tx, ty, bw, bh, obj, class)
                index = torch.tensor([grid_cell_xy[1], grid_cell_xy[0], adjusted_anchor_idx]).int()
                update = torch.cat((curr_box_xy, curr_box_wh, torch.tensor([1.0]).float(), curr_class), dim=0)

                # gird cell 하나에 대한 anchor 3개
                y[index[0]][index[1]][index[2]] = update

        return y

    def find_best_anchor(self, bbox):
        box_wh = torch.from_numpy(bbox[..., 2:4] - bbox[..., 0:2]).float()
        box_wh = box_wh.unsqueeze(1).repeat(1, anchors_wh.shape[0], 1).float()  # anchors_wh.shape[0] = 9

        intersection = torch.min(box_wh[..., 0], anchors_wh[..., 0]) * torch.min(box_wh[..., 1], anchors_wh[..., 1])
        box_area = box_wh[..., 0] * box_wh[..., 1]
        anchor_area = anchors_wh[..., 0] * anchors_wh[..., 1]

        iou = intersection / (box_area + anchor_area - intersection)
        anchor_idx = torch.argmax(iou, dim=-1).int()

        return anchor_idx

    def parse_y_features(self, idx):
        sizes = self.label_csv.iloc[idx, 1]
        sizes = list(map(int, sizes.split(' ')))

        bboxs = self.label_csv.iloc[idx, 2]
        bboxs = list(map(str, bboxs.split(',')))
        bbox = []
        for bx in bboxs:
            normal = list(map(int, bx.split(' ')))
            xmin = normal[0] / sizes[0]
            ymin = normal[1] / sizes[1]
            xmax = normal[2] / sizes[0]
            ymax = normal[3] / sizes[1]
            bbox.append([xmin, ymin, xmax, ymax])
        bbox = np.array(bbox)

        labels = torch.arange(self.num_classes)
        one_hot = torch.nn.functional.one_hot(labels)
        clss = self.label_csv.iloc[idx, 3]
        clss = list(map(int, clss.split(',')))
        cls = one_hot[clss]
        cls = np.array(cls)

        return bbox, cls

dataset = CustomDataset(20)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

print(dataloader)
# for i in range(len(dataset)):
#     sample = dataset[i]
#
#
#     if i == 5:
#         break


In [36]:
import torch
import pandas as pd
import os
import cv2
import numpy as np


anchors_wh = torch.tensor([[10, 13], [16, 30], [33, 23],
                           [30, 61], [62, 45], [59, 119],
                           [116, 90], [156, 198], [373, 326]]).float() / 416

label_csv = pd.read_csv(f'{csv_file}')

def parse_y_features(idx):
    sizes = label_csv.iloc[idx, 1]
    sizes = list(map(int, sizes.split(' ')))

    bboxs = label_csv.iloc[idx, 2]
    bboxs = list(map(str, bboxs.split(',')))
    bbox = []
    for bx in bboxs:
        normal = list(map(int, bx.split(' ')))
        xmin = normal[0] / sizes[0]
        ymin = normal[1] / sizes[1]
        xmax = normal[2] / sizes[0]
        ymax = normal[3] / sizes[1]

        bbox.append([xmin, ymin, xmax, ymax])
    bbox = np.array(bbox)

    labels = torch.arange(20)
    one_hot = torch.nn.functional.one_hot(labels)
    clss = label_csv.iloc[idx, 3]
    clss = list(map(int, clss.split(',')))
    cls = one_hot[clss]
    cls = np.array(cls)

    return bbox, cls

def preprocess_label_for_one_class(bbox, classes, grid_size, valid_anchor):
    y = torch.zeros((grid_size, grid_size, 3, 5 + 20))
    anchor_idx = find_best_anchor(bbox)

    num_boxes = classes.shape[0]
    for i in range(num_boxes):
        curr_class = torch.tensor(classes[i]).float()
        curr_box = torch.tensor(bbox[i]).float()
        curr_anchor = anchor_idx[i]

        # 앵커는 현재 그리드 사이즈에 맞는 것만 사용, 인덱스 3개로 조정
        if curr_anchor in valid_anchor:
            adjusted_anchor_idx = curr_anchor % 3

            # loss 계산을 위해 (xmin, ymin, xmax, ymax)를 (x, y, w, h)로 변환
            curr_box_xy = (curr_box[..., 0:2] + curr_box[..., 2:4]) / 2
            curr_box_wh = curr_box[..., 2:4] - curr_box[..., 0:2]

            # grid cell의 index
            grid_cell_xy = curr_box_xy // float(1 / grid_size)
            # gird[y][x][anchor] 형태 = (tx, ty, bw, bh, obj, class)
            index = torch.tensor([grid_cell_xy[1], grid_cell_xy[0], adjusted_anchor_idx.item()]).int()
            update = torch.cat((curr_box_xy, curr_box_wh, torch.tensor([1.0]).float(), curr_class), dim=0)
            print(f'index: {index}')
            print(f'update: {update}')
            # # gird cell 하나에 대한 anchor 3개
            y[index[0]][index[1]][index[2]] = update
    print(y.shape)

    return y

def find_best_anchor(bbox):
    box_wh = torch.from_numpy(bbox[..., 2:4] - bbox[..., 0:2]).float()
    box_wh = box_wh.unsqueeze(1).repeat(1, anchors_wh.shape[0], 1).float()  # anchors_wh.shape[0] = 9

    intersection = torch.min(box_wh[..., 0], anchors_wh[..., 0]) * torch.min(box_wh[..., 1], anchors_wh[..., 1])
    box_area = box_wh[..., 0] * box_wh[..., 1]
    anchor_area = anchors_wh[..., 0] * anchors_wh[..., 1]

    iou = intersection / (box_area + anchor_area - intersection)
    anchor_idx = torch.argmax(iou, dim=-1).int()

    return anchor_idx


for idx in range(1):
    img_path = os.path.join(f'{DB_path}', label_csv.iloc[idx, 0])
    image = cv2.resize(cv2.imread(img_path), (416, 416))
    image = torch.from_numpy(image).permute(2, 0, 1).float()

    bboxes, classes = parse_y_features(idx)

    label = (
        preprocess_label_for_one_class(bboxes, classes, 52, torch.tensor([0, 1, 2])),
        preprocess_label_for_one_class(bboxes, classes, 26, torch.tensor([3, 4, 5])),
        preprocess_label_for_one_class(bboxes, classes, 13, torch.tensor([6, 7, 8]))
    )

    print(type(label))



torch.Size([52, 52, 3, 25])
torch.Size([26, 26, 3, 25])
index: tensor([9, 4, 2], dtype=torch.int32)
update: tensor([0.3802, 0.7300, 0.7066, 0.5400, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
index: tensor([ 9, 10,  1], dtype=torch.int32)
update: tensor([0.8443, 0.7200, 0.3114, 0.5600, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
index: tensor([8, 1, 1], dtype=torch.int32)
update: tensor([0.1377, 0.6780, 0.2635, 0.6440, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
index: tensor([4, 2, 1], dtype=torch.int32)
update: tensor([0.1781, 0.3700, 0.3443, 0.7360, 1.0000, 0.0000, 0.

In [161]:
import torch
# torch.set_printoptions(edgeitems=2, threshold=50)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))



True
0
1
GeForce GTX 1650 Ti


In [35]:
y = torch.zeros((13, 13, 3, 5 + 20))
index= torch.tensor([7, 6, 1], dtype=torch.int32)
update = torch.tensor([0.5070, 0.5511, 0.3900, 0.5195, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
y[index[0]][index[1]][index[2]] = update
print(y.mean())
print(y[7][6])

tensor(0.0003)
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5070, 0.5511, 0.3900, 0.5195, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
